In [41]:
import gzip
import sys
from copy import deepcopy

sys.path.insert(0, './')
from ref_lib.GTF import GTFfile, GTFEntry, get_gtf_contents
from ref_lib.Fasta import FastaEntry, FastaFile

from collections import defaultdict, OrderedDict

It would be good to document this a bit more!

In [19]:
reverse_complement_dict = { "A" : "T",
                            "C" : "G",
                            "G" : "C",
                            "T" : "A",
                            "N" : "N"}

In [2]:
VCF_FIELDS = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER",  "INFO",    "FORMAT",  "CAST_EiJ"]

class VcfEntry:

    def __init__(self , vcf_line_contents ):
        assert len(vcf_line_contents) >= len(VCF_FIELDS)
        
        self.fields = { VCF_FIELDS[i] : vcf_line_contents[i] for i in range( len(VCF_FIELDS) ) }
        
        
    def __str__(self ):
        """
        This needs to be rewritten 
        """
        return "\t".join( [self.fields[f] for f in VCF_FIELDS] )


############################################################################
    
class VcfFile:
    '''
    This is a reader for 
    '''
    
    def __init__(self , file):
        myopen = open
        if file.endswith(".gz"):
            myopen = gzip.open

        if(file):
            self.f = myopen(file , "rt")
        else:
            self.f = stdin

    #####################################################

    def __enter__(self):
        return self

    #####################################################

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

    ######################################################

    def __getitem__(self, index):
        line = self.f.readline().strip()
        
        while line.startswith("#"):
            line = self.f.readline().strip()
        
        if line == "":
            raise IndexError
        #line_contents = line.split("\t")
        line_contents = line.split()
        if len(line_contents) < 9:
            raise IndexError
        return VcfEntry(line_contents)
                
    #########################################################

    def __del__(self):
        self.f.close()



In [23]:
vcf_file = "./reference_files/CAST.SNPs.validated.vcf.gz"

my_vcf = VcfFile(vcf_file)

In [4]:
gtf_file = "./reference_files/gencode.vM25.annotation.gtf.gz"

gtf_all = get_gtf_contents(gtf_file)

In [5]:
fasta_file   = "./reference_files/appris_mouse_v2_selected.fa.gz" 
fasta_reader = FastaFile(fasta_file)

In [6]:
fasta_entries = [a for a in fasta_reader]

In [42]:
fasta_dict = OrderedDict()

for f in fasta_entries:
    fasta_dict[f.header] = f.sequence

In [8]:
# We find the fasta entries of the transcript files

fasta_gtf_entries = {}
transcript_to_header = {}

for f in fasta_entries:
    contents                    = f.header.split("|")
    gene_id                     = contents[1].split(".")[0]
    transcript_id               = contents[0].split(".")[0]
    fasta_gtf_entries[f.header] = gtf_all[gene_id][transcript_id]
    
    transcript_to_header[transcript_id] = f.header
    
    

In [9]:
fasta_gtf_entries[ transcript_to_header['ENSMUST00000082419']  ]

{'exons': [(13552, 14070)],
 'CDS': [(13555, 14070, 0)],
 'strand': '-',
 'start': 13552,
 'end': 14070,
 'gene_type': 'protein_coding',
 'chr': 'chrM',
 'length': 519}

In [10]:
# Now we convert + strand coordinates
conversion_dict = dict()

for fasta_header, t_dict  in fasta_gtf_entries.items():
    
    if not conversion_dict.get(t_dict["chr"]):
        conversion_dict[t_dict["chr"]] = defaultdict(list)
        
    transcript_id = fasta_header.split("|")[0].split(".")[0]    
        
    t_position = 1
        
    if t_dict["strand"] == "+":
        
        temp_dict     = defaultdict(list) 
        
        for e in t_dict["exons"]:
            for i in range(e[0], e[1] + 1):
                conversion_dict[t_dict["chr"]][i].append( (transcript_id, t_position, "+" ) )
                t_position += 1
    else:
        for e in t_dict["exons"]:
            for i in range(e[1], e[0] - 1, -1):
                conversion_dict[t_dict["chr"]][i].append( (transcript_id, t_position, "-" ) )
                t_position += 1

In [11]:
for pos, mylist in conversion_dict['chr4'].items():
    if (len(mylist) > 1):
        print("{} : {}".format(pos, mylist) )

9449390 : [('ENSMUST00000038841', 1303, '+'), ('ENSMUST00000078139', 6383, '-')]
9449391 : [('ENSMUST00000038841', 1304, '+'), ('ENSMUST00000078139', 6382, '-')]
9449392 : [('ENSMUST00000038841', 1305, '+'), ('ENSMUST00000078139', 6381, '-')]
9449393 : [('ENSMUST00000038841', 1306, '+'), ('ENSMUST00000078139', 6380, '-')]
9449394 : [('ENSMUST00000038841', 1307, '+'), ('ENSMUST00000078139', 6379, '-')]
9449395 : [('ENSMUST00000038841', 1308, '+'), ('ENSMUST00000078139', 6378, '-')]
9449396 : [('ENSMUST00000038841', 1309, '+'), ('ENSMUST00000078139', 6377, '-')]
9449397 : [('ENSMUST00000038841', 1310, '+'), ('ENSMUST00000078139', 6376, '-')]
9449398 : [('ENSMUST00000038841', 1311, '+'), ('ENSMUST00000078139', 6375, '-')]
9449399 : [('ENSMUST00000038841', 1312, '+'), ('ENSMUST00000078139', 6374, '-')]
9449400 : [('ENSMUST00000038841', 1313, '+'), ('ENSMUST00000078139', 6373, '-')]
9449401 : [('ENSMUST00000038841', 1314, '+'), ('ENSMUST00000078139', 6372, '-')]
9449402 : [('ENSMUST00000038

41638149 : [('ENSMUST00000102963', 2501, '+'), ('ENSMUST00000127306', 1264, '-')]
41638150 : [('ENSMUST00000102963', 2502, '+'), ('ENSMUST00000127306', 1263, '-')]
41638151 : [('ENSMUST00000102963', 2503, '+'), ('ENSMUST00000127306', 1262, '-')]
41638152 : [('ENSMUST00000102963', 2504, '+'), ('ENSMUST00000127306', 1261, '-')]
41638153 : [('ENSMUST00000102963', 2505, '+'), ('ENSMUST00000127306', 1260, '-')]
41638154 : [('ENSMUST00000102963', 2506, '+'), ('ENSMUST00000127306', 1259, '-')]
41638155 : [('ENSMUST00000102963', 2507, '+'), ('ENSMUST00000127306', 1258, '-')]
41638156 : [('ENSMUST00000102963', 2508, '+'), ('ENSMUST00000127306', 1257, '-')]
41638157 : [('ENSMUST00000102963', 2509, '+'), ('ENSMUST00000127306', 1256, '-')]
41638158 : [('ENSMUST00000102963', 2510, '+'), ('ENSMUST00000127306', 1255, '-')]
43447724 : [('ENSMUST00000060864', 3607, '+'), ('ENSMUST00000107926', 1419, '-')]
43447725 : [('ENSMUST00000060864', 3608, '+'), ('ENSMUST00000107926', 1418, '-')]
43447726 : [('EN

59003210 : [('ENSMUST00000095070', 39, '+'), ('ENSMUST00000174664', 1, '+')]
59003211 : [('ENSMUST00000095070', 40, '+'), ('ENSMUST00000174664', 2, '+')]
59003212 : [('ENSMUST00000095070', 41, '+'), ('ENSMUST00000174664', 3, '+')]
59003213 : [('ENSMUST00000095070', 42, '+'), ('ENSMUST00000174664', 4, '+')]
59003214 : [('ENSMUST00000095070', 43, '+'), ('ENSMUST00000174664', 5, '+')]
59003215 : [('ENSMUST00000095070', 44, '+'), ('ENSMUST00000174664', 6, '+')]
59003216 : [('ENSMUST00000095070', 45, '+'), ('ENSMUST00000174664', 7, '+')]
59003217 : [('ENSMUST00000095070', 46, '+'), ('ENSMUST00000174664', 8, '+')]
59003218 : [('ENSMUST00000095070', 47, '+'), ('ENSMUST00000174664', 9, '+')]
59003219 : [('ENSMUST00000095070', 48, '+'), ('ENSMUST00000174664', 10, '+')]
59003220 : [('ENSMUST00000095070', 49, '+'), ('ENSMUST00000174664', 11, '+')]
59003221 : [('ENSMUST00000095070', 50, '+'), ('ENSMUST00000174664', 12, '+')]
59003222 : [('ENSMUST00000095070', 51, '+'), ('ENSMUST00000174664', 13, '

106747844 : [('ENSMUST00000047620', 1475, '+'), ('ENSMUST00000102762', 2400, '-')]
106747845 : [('ENSMUST00000047620', 1476, '+'), ('ENSMUST00000102762', 2399, '-')]
106747846 : [('ENSMUST00000047620', 1477, '+'), ('ENSMUST00000102762', 2398, '-')]
106747847 : [('ENSMUST00000047620', 1478, '+'), ('ENSMUST00000102762', 2397, '-')]
106747848 : [('ENSMUST00000047620', 1479, '+'), ('ENSMUST00000102762', 2396, '-')]
106747849 : [('ENSMUST00000047620', 1480, '+'), ('ENSMUST00000102762', 2395, '-')]
106747850 : [('ENSMUST00000047620', 1481, '+'), ('ENSMUST00000102762', 2394, '-')]
106747851 : [('ENSMUST00000047620', 1482, '+'), ('ENSMUST00000102762', 2393, '-')]
106747852 : [('ENSMUST00000047620', 1483, '+'), ('ENSMUST00000102762', 2392, '-')]
106747853 : [('ENSMUST00000047620', 1484, '+'), ('ENSMUST00000102762', 2391, '-')]
106747854 : [('ENSMUST00000047620', 1485, '+'), ('ENSMUST00000102762', 2390, '-')]
106747855 : [('ENSMUST00000047620', 1486, '+'), ('ENSMUST00000102762', 2389, '-')]
1067

118432909 : [('ENSMUST00000006557', 1767, '+'), ('ENSMUST00000006565', 1785, '-')]
118432910 : [('ENSMUST00000006557', 1768, '+'), ('ENSMUST00000006565', 1784, '-')]
118432911 : [('ENSMUST00000006557', 1769, '+'), ('ENSMUST00000006565', 1783, '-')]
118432912 : [('ENSMUST00000006557', 1770, '+'), ('ENSMUST00000006565', 1782, '-')]
118432913 : [('ENSMUST00000006557', 1771, '+'), ('ENSMUST00000006565', 1781, '-')]
118432914 : [('ENSMUST00000006557', 1772, '+'), ('ENSMUST00000006565', 1780, '-')]
118432915 : [('ENSMUST00000006557', 1773, '+'), ('ENSMUST00000006565', 1779, '-')]
118432916 : [('ENSMUST00000006557', 1774, '+'), ('ENSMUST00000006565', 1778, '-')]
118432917 : [('ENSMUST00000006557', 1775, '+'), ('ENSMUST00000006565', 1777, '-')]
118432918 : [('ENSMUST00000006557', 1776, '+'), ('ENSMUST00000006565', 1776, '-')]
118432919 : [('ENSMUST00000006557', 1777, '+'), ('ENSMUST00000006565', 1775, '-')]
118432920 : [('ENSMUST00000006557', 1778, '+'), ('ENSMUST00000006565', 1774, '-')]
1184

139603254 : [('ENSMUST00000040023', 434, '+'), ('ENSMUST00000178644', 69, '+')]
139603255 : [('ENSMUST00000040023', 435, '+'), ('ENSMUST00000178644', 70, '+')]
139603256 : [('ENSMUST00000040023', 436, '+'), ('ENSMUST00000178644', 71, '+')]
139603257 : [('ENSMUST00000040023', 437, '+'), ('ENSMUST00000178644', 72, '+')]
139603258 : [('ENSMUST00000040023', 438, '+'), ('ENSMUST00000178644', 73, '+')]
139603259 : [('ENSMUST00000040023', 439, '+'), ('ENSMUST00000178644', 74, '+')]
139603260 : [('ENSMUST00000040023', 440, '+'), ('ENSMUST00000178644', 75, '+')]
139603261 : [('ENSMUST00000040023', 441, '+'), ('ENSMUST00000178644', 76, '+')]
139603262 : [('ENSMUST00000040023', 442, '+'), ('ENSMUST00000178644', 77, '+')]
139603263 : [('ENSMUST00000040023', 443, '+'), ('ENSMUST00000178644', 78, '+')]
139603264 : [('ENSMUST00000040023', 444, '+'), ('ENSMUST00000178644', 79, '+')]
139603265 : [('ENSMUST00000040023', 445, '+'), ('ENSMUST00000178644', 80, '+')]
139603266 : [('ENSMUST00000040023', 446,

139662122 : [('ENSMUST00000178644', 4265, '+'), ('ENSMUST00000030510', 1446, '+')]
139662123 : [('ENSMUST00000178644', 4266, '+'), ('ENSMUST00000030510', 1447, '+')]
139662124 : [('ENSMUST00000178644', 4267, '+'), ('ENSMUST00000030510', 1448, '+')]
139662125 : [('ENSMUST00000178644', 4268, '+'), ('ENSMUST00000030510', 1449, '+')]
139662126 : [('ENSMUST00000178644', 4269, '+'), ('ENSMUST00000030510', 1450, '+')]
139662127 : [('ENSMUST00000178644', 4270, '+'), ('ENSMUST00000030510', 1451, '+')]
139662128 : [('ENSMUST00000178644', 4271, '+'), ('ENSMUST00000030510', 1452, '+')]
139662129 : [('ENSMUST00000178644', 4272, '+'), ('ENSMUST00000030510', 1453, '+')]
139662130 : [('ENSMUST00000178644', 4273, '+'), ('ENSMUST00000030510', 1454, '+')]
139662131 : [('ENSMUST00000178644', 4274, '+'), ('ENSMUST00000030510', 1455, '+')]
139662132 : [('ENSMUST00000178644', 4275, '+'), ('ENSMUST00000030510', 1456, '+')]
139662133 : [('ENSMUST00000178644', 4276, '+'), ('ENSMUST00000030510', 1457, '+')]
1396

148615114 : [('ENSMUST00000052060', 2673, '+'), ('ENSMUST00000084125', 4745, '-')]
148615115 : [('ENSMUST00000052060', 2674, '+'), ('ENSMUST00000084125', 4744, '-')]
148615116 : [('ENSMUST00000052060', 2675, '+'), ('ENSMUST00000084125', 4743, '-')]
148615117 : [('ENSMUST00000052060', 2676, '+'), ('ENSMUST00000084125', 4742, '-')]
148615118 : [('ENSMUST00000052060', 2677, '+'), ('ENSMUST00000084125', 4741, '-')]
148615119 : [('ENSMUST00000052060', 2678, '+'), ('ENSMUST00000084125', 4740, '-')]
148615120 : [('ENSMUST00000052060', 2679, '+'), ('ENSMUST00000084125', 4739, '-')]
148615121 : [('ENSMUST00000052060', 2680, '+'), ('ENSMUST00000084125', 4738, '-')]
148615122 : [('ENSMUST00000052060', 2681, '+'), ('ENSMUST00000084125', 4737, '-')]
148615123 : [('ENSMUST00000052060', 2682, '+'), ('ENSMUST00000084125', 4736, '-')]
148615124 : [('ENSMUST00000052060', 2683, '+'), ('ENSMUST00000084125', 4735, '-')]
148615125 : [('ENSMUST00000052060', 2684, '+'), ('ENSMUST00000084125', 4734, '-')]
1486

In [24]:
counter = 0

f_handle = gzip.open("transcriptomic_variants.vcf.gz", "wt")

for v in my_vcf:
    if conversion_dict[v.fields['CHROM']].get( int(v.fields['POS']) ):
        counter +=1
        
        for entry in  conversion_dict[v.fields['CHROM']][ int(v.fields['POS']) ]: 
            this_entry = deepcopy(v)
            this_entry.fields['CHROM'] = transcript_to_header[entry[0]]
            this_entry.fields['POS']   = str(entry[1])
            
            if entry[2] == "-":
                this_entry.fields['REF'] =  "".join(map( lambda x: reverse_complement_dict[x], this_entry.fields['REF']))
                this_entry.fields['ALT'] =  "".join(map( lambda x: reverse_complement_dict[x], this_entry.fields['ALT']))
                
            print(this_entry, file = f_handle)
            
 
f_handle.close()
        
print("Variants found {}".format(counter))

Variants found 207451


In [28]:
this_header = "ENSMUST00000215295.1|ENSMUSG00000096054.3|OTTMUSG00000063506.2|OTTMUST00000154809.1|Syne1-205|Syne1|27908|UTR5:1-418|CDS:419-26818|UTR3:26819-27908|"

fasta_dict[this_header][17853]

'G'

In [29]:
fasta_dict[this_header]

'...'

In [43]:
## Sanity_cehck
## Are reference nucleotides in the Vcf file same as the nucleotides in the fasta files?

produced_vcf_file = "transcriptomic_variants.vcf.gz"
#produced_vcf_file = "mock.vcf.gz"

produced_vcf = VcfFile(produced_vcf_file)

In [44]:
# This should NOT print anything on the produced_vcf_file
# We put one fault record on the mock file to test our test :)

for v in produced_vcf:
    observed_nuc = v.fields["REF"]
    
    # Note that vcf is 1-based and lists are 0-based
    # so we need -1 in the index
    expected_nuc = fasta_dict[v.fields["CHROM"]][ int(v.fields["POS"])-1]
    
    if observed_nuc != expected_nuc:
        print("In", v.fields["CHROM"])
        print("At position ", v.fields["POS"])
        print("{} != {}".format(observed_nuc, expected_nuc) )

In [55]:
## Now we mask our Transcriptomic fasta file based on the variants on VCF

produced_vcf = VcfFile(produced_vcf_file)

masked_fasta_dict = deepcopy(fasta_dict)

for v in produced_vcf:
    #print(v.fields["POS"])
    #print(v.fields["CHROM"])
    #print(masked_fasta_dict[v.fields["CHROM"]])
    masked_sequence = masked_fasta_dict[v.fields["CHROM"]][ :(int(v.fields["POS"]) - 1)  ] + \
                      "N" + \
                      masked_fasta_dict[v.fields["CHROM"]][ int(v.fields["POS"]): ]
    masked_fasta_dict[v.fields["CHROM"]] = masked_sequence
    


with gzip.open("variant_masked_mouse_transcriptome.fa.gz", "wt") as out_stream:
    for header, sequence in masked_fasta_dict.items():
        this_entry = FastaEntry(header = header, sequence = sequence)
        print(this_entry, file = out_stream)
        
    

In [56]:
# Let's make sure that all masked positions are N's

masked_fasta_file   = "variant_masked_mouse_transcriptome.fa.gz"
masked_fasta_reader  = FastaFile(masked_fasta_file)

read_fasta_dict = {}

for f in masked_fasta_reader:
    read_fasta_dict[f.header] = f.sequence

produced_vcf_file = "transcriptomic_variants.vcf.gz"
#produced_vcf_file = "mock.vcf.gz"

produced_vcf = VcfFile(produced_vcf_file)

for v in produced_vcf:
    
    # Note that vcf is 1-based and lists are 0-based
    # so we need -1 in the index
    assert read_fasta_dict[v.fields["CHROM"]][ int(v.fields["POS"])-1] == "N"
